In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets.mnist import load_data
import numpy as np
import matplotlib.pyplot as plt


In [3]:
def define_discirminator(in_shape = (28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2,2), padding = 'same', input_shape = in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(learning_rate = 0.0002, beta_1=0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])
    
    return model

In [4]:
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 128*7*7
    
    model.add(Dense(n_nodes, input_dim = latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    
    return model

In [5]:
def define_GAN(g_model, d_model):
    d_model.trainable = False
    
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    
    opt = Adam(learning_rate = 0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model

In [6]:
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = np.expand_dims(trainX, axis = -1)
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X

In [7]:
def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))
    return X, y

In [8]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples = 100):
    x_real, y_real = generate_real_samples(dataset, n_samples)    
    _, acc_real = d_model.evaluate(x_real, y_real, verbose=0)
    
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = g_model.evaluate(x_fake, y_fake, verbose=0)
    
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    
    save_plot(x_fake, epoch)
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename) 

In [11]:
def save_plot(examples, epoch, n=10):
 # plot images
 for i in range(n * n):
     # define subplot
     plt.subplot(n, n, 1 + i)
     # turn off axis
     plt.axis('off')
     # plot raw pixel data
     plt.imshow(examples[i, :, :, 0], cmap='gray_r')
     # save plot to file
     filename = 'generated_plot_e%03d.png' % (epoch+1)
     plt.savefig(filename)
     plt.close()

In [12]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch/2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            x_real, y_real = generate_real_samples(dataset, half_batch)
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            x, y = np.vstack((x_real, x_fake)), np.vstack((y_real, y_fake))
            
            d_loss, _ = d_model.train_on_batch(x, y)
            x_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))
            
            g_loss = gan_model.train_on_batch(x_gan, y_gan)
            
            print(">%d, %d/%d, d=%.3f, g=%.3f" % (i+1, j+1, bat_per_epo, d_loss, g_loss))
            
            if (i+1)%10 == 0:
                summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [13]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discirminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_GAN(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

4/4 [==============================] - 4s 4ms/step
>1, 1/234, d=0.691, g=0.736
4/4 [==============================] - 0s 4ms/step
>1, 2/234, d=0.681, g=0.748
4/4 [==============================] - 0s 4ms/step


KeyboardInterrupt: 